In [1]:
import os
import numpy as np
import pandas as pd
import gc

In [2]:
!ls ../input/hm-public-submissions

0-0226-byfone-chris-combination-approach.csv
h-m-trending-products-weekly-add-test.csv
hnm-exponential-decay-with-alternate-items.csv


# **Adding LSTM models to the ensemble**

# Note
* I am not sure whether I should get any credit for this notebook as this is based(mostly) on the work of others.
* I added two more submissions to the ensemble and tried a few options with them.

# New Notebooks
* LB: 0.0210 - https://www.kaggle.com/astrung/recbole-lstm-sequential-for-recomendation-tutorial
* LB: 0.0221 - https://www.kaggle.com/astrung/lstm-sequential-modelwith-item-features-tutorial
* both notebooks by @astrung

# Predictions in this competition are a list of 12 itens ordered by most relevant first.
# In this notebook I will show how to ensemble lists of different models
# To ensemble I used submissions from 3 public notebooks:
- LB: 0.0225 - https://www.kaggle.com/lichtlab/0-0226-byfone-chris-combination-approach/data?scriptVersionId=89289696
- LB: 0.0225 - https://www.kaggle.com/lunapandachan/h-m-trending-products-weekly-add-test/notebook
- LB: 0.0217 - https://www.kaggle.com/tarique7/hnm-exponential-decay-with-alternate-items/notebook

In [3]:
sub0 = pd.read_csv('../input/hm-public-submissions/0-0226-byfone-chris-combination-approach.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('../input/hm-public-submissions/h-m-trending-products-weekly-add-test.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('../input/hm-public-submissions/hnm-exponential-decay-with-alternate-items.csv').sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape

((1371980, 2), (1371980, 2), (1371980, 2))

In [4]:
sub3 = pd.read_csv('../input/submission-recbole-lstm/submission.csv').sort_values('customer_id').reset_index(drop=True)

In [5]:
sub4 = pd.read_csv('../input/submission-lstm-sequential/submission (1).csv').sort_values('customer_id').reset_index(drop=True)

In [6]:
# How many predictions are in common between models

print((sub0['prediction']==sub1['prediction']).mean())
print((sub0['prediction']==sub2['prediction']).mean())
print((sub1['prediction']==sub2['prediction']).mean())

0.4324239420399714
0.0005991340981646963
0.0005991340981646963


In [7]:
# How many predictions are in common between old models and new 

print((sub3['prediction']==sub0['prediction']).mean())
print((sub3['prediction']==sub1['prediction']).mean())
print((sub3['prediction']==sub2['prediction']).mean())

0.0065190454671350895
0.006790915319465299
0.00031268677385967725


In [8]:
# How many predictions are in common between old models and new

print((sub4['prediction']==sub0['prediction']).mean())
print((sub4['prediction']==sub1['prediction']).mean())
print((sub4['prediction']==sub2['prediction']).mean())
print((sub4['prediction']==sub3['prediction']).mean())

0.007623288969226956
0.007895158821557165
0.00043878190644178487
0.9716358839050132


In [9]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
sub0['prediction3'] = sub3['prediction']
sub0['prediction4'] = sub4['prediction']
del sub1, sub2, sub3, sub4
gc.collect()
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3,prediction4
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,0568601043 0568601006 0656719005 0745232001 07...,0568601043 0924243001 0924243002 0918522001 07...,0568601043 0568601006 0656719005 0745232001 09...,0568601043 0568601006 0656719005 0745232001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,0826211002 0800436010 0739590027 0723529001 08...,0924243001 0924243002 0918522001 0751471001 04...,0826211002 0800436010 0924243001 0739590027 07...,0826211002 0800436010 0924243001 0739590027 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0924243001 0924243002 0918522001 07...,0794321007 0852643001 0852643003 0858883002 09...,0794321007 0852643001 0852643003 0858883002 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,0448509014 0573085028 0751471001 0706016001 06...,0924243001 0924243002 0918522001 0751471001 04...,0448509014 0573085028 0924243001 0751471001 07...,0448509014 0573085028 0924243001 0751471001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,0730683050 0791587015 0896152002 0818320001 09...,0924243001 0924243002 0918522001 0751471001 04...,0730683050 0791587015 0924243001 0896152002 08...,0730683050 0791587015 0924243001 0896152002 08...


In [10]:
def cust_blend(dt, W = [1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    
    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

In [11]:
sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95,0.85], axis=1)
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3,prediction4,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,0568601043 0568601006 0656719005 0745232001 07...,0568601043 0924243001 0924243002 0918522001 07...,0568601043 0568601006 0656719005 0745232001 09...,0568601043 0568601006 0656719005 0745232001 09...,0568601043 0568601006 0656719005 0745232001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,0826211002 0800436010 0739590027 0723529001 08...,0924243001 0924243002 0918522001 0751471001 04...,0826211002 0800436010 0924243001 0739590027 07...,0826211002 0800436010 0924243001 0739590027 07...,0826211002 0800436010 0924243001 0739590027 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0924243001 0924243002 0918522001 07...,0794321007 0852643001 0852643003 0858883002 09...,0794321007 0852643001 0852643003 0858883002 09...,0794321007 0852643001 0852643003 0858883002 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,0448509014 0573085028 0751471001 0706016001 06...,0924243001 0924243002 0918522001 0751471001 04...,0448509014 0573085028 0924243001 0751471001 07...,0448509014 0573085028 0924243001 0751471001 07...,0448509014 0573085028 0924243001 0751471001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,0730683050 0791587015 0896152002 0818320001 09...,0924243001 0924243002 0918522001 0751471001 04...,0730683050 0791587015 0924243001 0896152002 08...,0730683050 0791587015 0924243001 0896152002 08...,0730683050 0791587015 0924243001 0896152002 08...


In [12]:
# How many predictions are in common with ensemble

print((sub0['prediction']==sub0['prediction0']).mean())
print((sub0['prediction']==sub0['prediction1']).mean())
print((sub0['prediction']==sub0['prediction2']).mean())
print((sub0['prediction']==sub0['prediction3']).mean())
print((sub0['prediction']==sub0['prediction4']).mean())

0.006508841236752722
0.006780711089082931
0.0003163311418533798
0.9707145876762052
0.9707175031706001


**The results of comparison seem interesting**

# Make a submission

In [13]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
del sub0['prediction3']
del sub0['prediction4']
gc.collect()
sub0.to_csv('submission.csv', index=False)